# Работа с Excel

Материалы:
* Макрушин С.В. Лекция 7: Работа с Excel
* https://docs.xlwings.org/en/stable/quickstart.html
* https://nbviewer.jupyter.org/github/pybokeh/jupyter_notebooks/blob/master/xlwings/Excel_Formatting.ipynb#search_text


In [228]:
from pathlib import Path

import pandas as pd
import xlwings as xw
from xlwings.constants import BordersIndex, BorderWeight, HAlign, RgbColor

In [229]:
DATA_DIR = Path('data/')
OUTPUT_DIR = Path('output/')

In [230]:
def get_new_col_idx(sheet):
    return sheet.range('A1').expand('right').shape[1] + 1


def get_last_row_idx(sheet):
    return sheet.range('A1').expand('down').shape[0]


def format_header(cell):
    sheet.range(cell).api.Font.Bold = True
    sheet.range(cell).api.HorizontalAlignment = HAlign.xlHAlignCenter
    for col_idx in (
            BordersIndex.xlEdgeTop,
            BordersIndex.xlEdgeRight,
            BordersIndex.xlEdgeBottom,
            BordersIndex.xlEdgeLeft
    ):
        sheet.range(cell).api.Borders(col_idx).LineStyle = 1
        sheet.range(cell).api.Borders(col_idx).Weight = BorderWeight.xlThin

## Задачи для совместного разбора

1. На листе "Рецептура" файла `себестоимостьА_в1.xlsx` для области "Пшеничный хлеб" рассчитать себестоимость всех видов продукции.

2. Результаты расчетов 1.1 сохранить в отдельном столбце области "Пшеничный хлеб"

3. Приблизить форматирование столбца, добавленного в задаче 2 к оформлению всей области.

4. Выполнить 3 с помощью "протягиваемых" формул.

## Лабораторная работа 7

1. Загрузите данные из файлов `reviews_sample.csv` (__ЛР2__) и `recipes_sample_with_tags_ingredients.csv` (__ЛР5__) в виде `pd.DataFrame`. Обратите внимание на корректное считывание столбца(ов) с индексами. Оставьте в таблице с рецептами следующие столбцы: `id`, `name`, `minutes`, `submitted`, `description`, `n_ingredients`

In [231]:
reviews_df = pd.read_csv(
    DATA_DIR.joinpath('reviews_sample.csv'),
    sep=',',
    index_col=0,
    parse_dates=['date']
)
reviews_df.head()

,user_id,recipe_id,date,rating,review
370476,21752,57993,2003-05-01,5,Last week whole sides of frozen salmon fillet ...
624300,431813,142201,2007-09-16,5,So simple and so tasty! I used a yellow capsi...
187037,400708,252013,2008-01-10,4,"Very nice breakfast HH, easy to make and yummy..."
706134,2001852463,404716,2017-12-11,5,These are a favorite for the holidays and so e...
312179,95810,129396,2008-03-14,5,Excellent soup! The tomato flavor is just gre...


In [232]:
recipes_df = pd.read_csv(
    DATA_DIR.joinpath('recipes_sample_with_filled_nsteps.csv'),
    sep=',',
    usecols=['id', 'name', 'minutes', 'submitted', 'description', 'n_ingredients'],
    parse_dates=['submitted']
)
recipes_df.head()

,name,id,minutes,submitted,description,n_ingredients
0,george s at the cove black bean soup,44123,90,2002-10-25,an original recipe created by chef scott meska...,18.0
1,healthy for them yogurt popsicles,67664,10,2003-07-26,my children and their friends ask for my homem...,NaN
2,i can t believe it s spinach,38798,30,2002-08-29,"these were so go, it surprised even me.",8.0
3,italian gut busters,35173,45,2002-07-27,my sister-in-law made these for us at a family...,NaN
4,love is in the air beef fondue sauces,84797,25,2004-02-23,i think a fondue is a very romantic casual din...,NaN


2. Случайным образом выберите 5% строк из каждой таблицы и сохраните две таблицы на разные листы в один файл `recipes.xlsx`. Дайте листам названия "Рецепты" и "Отзывы", соответствующие содержанию таблиц. 

In [233]:
reviews_sample = reviews_df.sample(frac=0.05)
reviews_sample

,user_id,recipe_id,date,rating,review
995774,280271,382536,2012-10-07,4,I loved it...but DH didn't...he really didn't ...
134625,830622,74007,2011-01-03,4,"Okay, even though I messed up on this pie- usi..."
309425,241948,95680,2007-01-13,5,Made these with family friends. GREAT for kids...
142633,369386,257172,2007-11-15,5,These were funny! I got a cat and a bat in my ...
187543,102519,246,2004-08-01,3,This is a fairly good dip but it needed more f...
...,...,...,...,...,...
474584,55729,86129,2005-07-24,4,"On a recent health kick, I bought a 2 lb. bag ..."
990923,1518560,226555,2010-01-21,5,I LOVE this recipe! I just made it tonight fo...
852589,37449,413824,2010-03-24,5,Oh yummy yum yum! I used Asian Chili sauce and...
101310,68460,43901,2004-10-31,5,I can't believe this hasn't been rated yet! T...


In [234]:
recipes_sample = recipes_df.sample(frac=0.05)
recipes_sample

,name,id,minutes,submitted,description,n_ingredients
13091,ground beef stroganoff,123503,15,2005-05-24,this is one of my mother's recipes that i've m...,10.0
21678,pumpkin coffee cake with brown sugar glaze,484133,40,2012-07-24,found on pinterest. looks delicious! i am go...,12.0
3670,bread machine naan,303556,90,2008-05-14,i was looking for a recipe for naan that i cou...,NaN
4545,candied yams with apples,74509,95,2003-10-30,this sounds so good. i got the recipe out of a...,8.0
20621,pesto mozzarella skewers,203322,30,2007-01-04,a beautiful and tasty way to use the widely av...,8.0
...,...,...,...,...,...,...
11631,fruit honey and coconut energy bars,94905,15,2004-07-02,why pay almost $3 or $4 for an overpriced box ...,11.0
2213,banana walnut and date loaf,326718,75,2008-09-22,great for the lunch box! this loaf can be topp...,9.0
14633,italian chicken sandwiches,185051,13,2006-09-07,"got this from the healthy, hearty cookbook. so...",5.0
22006,quick tasty tuna wraps,302932,10,2008-05-09,i created this when i didn't have alot of mone...,6.0


In [235]:
with pd.ExcelWriter(OUTPUT_DIR.joinpath('recipes.xlsx')) as writer:
    reviews_sample.to_excel(writer, sheet_name='Отзывы', index=False)
    recipes_sample.to_excel(writer, sheet_name='Рецепты', index=False)

3. Используя `xlwings`, добавьте на лист `Рецепты` столбец `seconds_assign`, показывающий время выполнения рецепта в секундах. Выполните задание при помощи присваивания массива значений диапазону ячеек.

In [236]:
wb = xw.Book(OUTPUT_DIR.joinpath('recipes.xlsx'))

In [237]:
sheet = wb.sheets['Рецепты']

In [238]:
seconds_assign = (recipes_sample['minutes'] * 60).rename('seconds_assign')
cell = (1, get_new_col_idx(sheet))
sheet.range(cell).options(index=False).value = seconds_assign
format_header(cell)

4. Используя `xlwings`, добавьте на лист `Рецепты` столбец `seconds_formula`, показывающий время выполнения рецепта в секундах. Выполните задание при помощи формул Excel.

In [239]:
formula = '=C2 * 60'
col_idx = get_new_col_idx(sheet)
last_row_idx = get_last_row_idx(sheet)

sheet.range(1, col_idx).value = 'seconds_formula'
sheet.range((2, col_idx), (last_row_idx, col_idx)).formula = formula
format_header((1, col_idx))

5. Добавьте на лист `Рецепты`  столбец `n_reviews`, содержащий кол-во отзывов для этого рецепта. Выполните задание при помощи формул Excel.

In [240]:
formula = '=COUNTIF(Отзывы!B:B, "="&B2)'  # чем я занимаюсь -_- ?????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????
col_idx = get_new_col_idx(sheet)
last_row_idx = get_last_row_idx(sheet)

sheet.range(1, col_idx).value = 'n_reviews'
sheet.range((2, col_idx), (last_row_idx, col_idx)).formula = formula
format_header((1, col_idx))

6. Сделайте названия всех добавленных столбцов полужирными и выровняйте по центру ячейки.

In [241]:
[*'Уже']

['У', 'ж', 'е']

In [242]:
sheet.range('A1').expand('right').autofit()

7. Раскрасьте ячейки столбца `minutes` в соответствии со следующим правилом: если рецепт выполняется быстрее 5 минут, то цвет - зеленый; от 5 до 10 минут - жёлтый; и больше 10 - красный.

In [243]:
last_row_idx = get_last_row_idx(sheet)
values = sheet.range(f'C2:C{last_row_idx}').value  # так быстрее
for i in range(last_row_idx - 1):
    # кажется это кейс для match-case, нужно ставить python 3.10
    minutes = values[i]
    if minutes < 5:
        color = RgbColor.rgbLimeGreen
    elif 5 <= minutes <= 10:
        color = RgbColor.rgbYellow
    else:
        color = RgbColor.rgbRed
    sheet.range(f'C{i + 2}').color = color

# 1500 строчек, как же д-о-ооооооооооооооооо-о-лго

8. Напишите функцию `validate()`, которая проверяет соответствие всех строк из листа `Отзывы` следующим правилам:
    * Рейтинг - это число от 0 до 5 включительно
    * Соответствующий рецепт имеется на листе `Рецепты`
    
В случае несоответствия этим правилам, выделите строку красным цветом

In [244]:
import numpy as np


def random_color(n: int) -> int:
    colors = list(map(
        lambda x: x[1],
        filter(lambda x: x[0].startswith('rgb') and 'White' not in x[0], RgbColor.__dict__.items())
    ))
    colors_sample_idx = np.random.default_rng().integers(0, len(colors), n)
    i = 0
    while True:
        yield colors[colors_sample_idx[i % n]]
        i += 1

In [245]:
import timeit


def validate(wb: xw.Book) -> None:
    reviews_sheet = wb.sheets['Отзывы']
    recipes_sheet = wb.sheets['Рецепты']

    def invalid_recipe_id() -> pd.Series:
        reviews_recipe_id_range = find_col(reviews_sheet, 'recipe_id')
        recipes_id_range = find_col(recipes_sheet, 'id')

        reviews_recipe_id_series = pd.Series(reviews_recipe_id_range.value[1:], dtype=int)
        recipes_id_series = pd.Series(recipes_id_range.value[1:], dtype=int)

        return reviews_recipe_id_series[~reviews_recipe_id_series.isin(recipes_id_series)]

    def invalid_rating() -> pd.Series:
        rating_range = find_col(reviews_sheet, 'rating')
        rating_series = pd.Series(rating_range.value[1:], dtype=int)

        return rating_series[(5 < rating_series) | (rating_series < 0)]

    def format_bad(*list_bad_indexes) -> None:
        last_col_idx = reviews_sheet.range('A1').expand('right').shape[1]
        color_generator = random_color(len(list_bad_indexes[0]))
        for bad_indexes in list_bad_indexes:
            for i in bad_indexes:
                row_i = i + 2
                reviews_sheet.range((row_i, 1), (row_i, last_col_idx)).color = next(color_generator)

    start = timeit.default_timer()
    recipe_id_bad_indexes = invalid_recipe_id().index
    print(f'recipe_id_bad_indexes: {timeit.default_timer() - start}')

    start = timeit.default_timer()
    rating_bad_indexes = invalid_rating().index
    print(f'rating_bad_indexes: {timeit.default_timer() - start}')

    start = timeit.default_timer()
    format_bad(recipe_id_bad_indexes, rating_bad_indexes)
    print(f'format_bad: {timeit.default_timer() - start}')


def find_col(sheet: xw.Sheet, name: str) -> xw.Range:
    headers: list = sheet.range('A1').expand('right').value
    col_idx = headers.index(name) + 1
    return sheet.range(1, col_idx).expand('down')

In [246]:
validate(wb)

recipe_id_bad_indexes: 0.08959409999988566
rating_bad_indexes: 0.04636960000061663
format_bad: 15.652697099998477


In [247]:
wb.save()
wb.close()
